# ДЗ-3: Трекинг LLM-инференса с vLLM

Собираем образ mlflow из mlflow.dockerfile

In [1]:
%%sh

docker build -f mlflow.dockerfile -t mlflow:1.0 .

#0 building with "default" instance using docker driver

#1 [internal] load build definition from mlflow.dockerfile
#1 transferring dockerfile: 201B 0.0s done
#1 DONE 0.1s

#2 [internal] load metadata for docker.io/library/python:3.9.13-slim
#2 DONE 2.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/3] FROM docker.io/library/python:3.9.13-slim@sha256:dcf2eafca55558d8b1aa73edd6aa41b7187c5bcb63e533a7b04a0673f81f37fe
#4 resolve docker.io/library/python:3.9.13-slim@sha256:dcf2eafca55558d8b1aa73edd6aa41b7187c5bcb63e533a7b04a0673f81f37fe 0.0s done
#4 sha256:b94fc7ac342a843369c0eaa335613ab9b3761ff5ddfe0217a65bfd3678614e22 0B / 11.59MB 0.1s
#4 sha256:dcf2eafca55558d8b1aa73edd6aa41b7187c5bcb63e533a7b04a0673f81f37fe 1.86kB / 1.86kB done
#4 sha256:b3bb514565b331c9a6f36c396af2a937f730b64f34004b750a3b3d5874abd990 1.37kB / 1.37kB done
#4 sha256:5da6ce3c33c6783782eece610be5c83a34001d6cd1d66ea32381afac543f366b 7.48kB / 7.48kB done
#4 sha256:7a6db449b51b92eac5c8

In [ ]:
%%sh

docker run --rm  -p 5000:5000 --name=mlflow -v mlflow:/app mlflow:1.0


Запускаем vLLM c моделью facebook/opt-1.3b

In [ ]:
%%sh

docker run --rm --runtime nvidia --gpus all \
    -v ~/.cache/huggingface:/root/.cache/huggingface \
    --env "HUGGING_FACE_HUB_TOKEN=<secret>
    --ipc=host \
    vllm/vllm-openai:latest \
    --model facebook/opt-1.3b

Проверяем работоспособность контейнера

In [23]:
%%sh

curl -X POST "http://localhost:8000/v1/completions" \
	-H "Content-Type: application/json" \
	--data '{
		"model": "facebook/opt-1.3b",
		"prompt": "What is the capital of Germany?",
		"max_tokens": 50,
		"temperature": 0.1
	}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   539  100   415  100   124   3612   1079 --:--:-- --:--:-- --:--:--     0--:-- --:--:--  468615  100   124   3610   1078 --:--:-- --:--:-- --:--:--  4686


{"id":"cmpl-4dd60793b15a4a678a63acf2b8d8c2ca","object":"text_completion","created":1756652543,"model":"facebook/opt-1.3b","choices":[{"index":0,"text":"\nBerlin.","logprobs":null,"finish_reason":"stop","stop_reason":null,"prompt_logprobs":null}],"service_tier":null,"system_fingerprint":null,"usage":{"prompt_tokens":8,"total_tokens":13,"completion_tokens":5,"prompt_tokens_details":null},"kv_transfer_params":null}

Проверяем с помощью OpenAI

модель facebook/opt-1.3b  не поддерживает тип запроса Chat - используем Completion

In [24]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="",
)

completion = client.completions.create(
    model="facebook/opt-1.3b",
    prompt="What is the capital of Germany?",
    max_tokens=50,
    temperature=0.1
)

print(completion.choices[0].text.strip())




Berlin.


Добавляем mlflow

In [25]:
%%sh
pip install mlflow

  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0[0━ 0.0/26.4 MB ? eta -:--:--━━━━━━━━━━━━━━━━━ 0.8/26.4 MB 6.3 MB/s eta 0:00:05━━━━━━━━━━━━━━━━━ 1.8/26.4 MB 7.6 MB/s eta 0:00:04━━━━━━━━━━━━━━━━━ 4.2/26.4 MB 8.4 MB/s eta 0:00:03━━━━━━━━━━━━━━━━━ 6.0/26.4 MB 8.3 MB/s eta 0:00:03━━━━━━━━━━━━━━━━━ 6.8/26.4 MB 7.4 MB/s eta 0:00:03━━━━━━━━━━━━━━━━━ 8.1/26.4 MB 7.2 MB/s eta 0:00:03━━━━━━━━━━━━━━━━━ 9.4/26.4 MB 7.3 MB/s eta 0:00:03━━━━━━━━━━━━━━━━━ 10.7/26.4 MB 7.1 MB/s eta 0:00:03━━━━━━━━━━━━━━━━━ 12.3/26.4 MB 7.1 MB/s eta 0:00:02━━━━━━━━━━━━━━━━━ 13.6/26.4 MB 7.1 MB/s eta 0:00:02━━━━━━━━━━━━━━━━━ 14.9/26.4 MB 7.0 MB/s eta 0:00:02m━━━━━━━━━━━━━━━ 16.3/26.4 MB 7.1 MB/s eta 0:00:02╺━━━━━━━━━━━━━ 17.3/26.4 MB 6.9 MB/s eta 0:00:02m╸━━━━━━━━━━━ 18.9/26.4 MB 6.9 MB/s eta 0:00:02━╸━━━━━━━━━ 20.4/26.4 MB 7.0 MB/s eta 0:00:01━━━╺━━━━━━━ 21.2/26.4 MB 6.9 MB/s eta 0:00:01━━━━╸━━━━━━ 22.3/26.4 MB 6.7 MB/s eta 0:00:01━━━━━━╺━━━━ 23.3/26.4 MB 6.7 MB/s eta 0:0

Инициализируем MLFlow

In [26]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("vLLM_inference_metrics")

2025/08/31 18:10:20 INFO mlflow.tracking.fluent: Experiment with name 'vLLM_inference_metrics' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/506265846202140729', creation_time=1756653020675, experiment_id='506265846202140729', last_update_time=1756653020675, lifecycle_stage='active', name='vLLM_inference_metrics', tags={}>

Создаем набор метрик

In [32]:
def make_genai_metric(response):
    metrics = {
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens,
    }
    return metrics

Подключаем экспорт метрик в запросы к модели

In [33]:
import time


with mlflow.start_run():
    start_time = time.time()

    completion = client.completions.create(
        model="facebook/opt-1.3b",
        prompt="What is the capital of Germany?",
        max_tokens=50,
        temperature=0.1
    )

    end_time = time.time()
    duration = end_time - start_time

    # Добавление времени выполнения как метрики
    mlflow.log_metric("inference_duration_seconds", duration)

    # Логирование метрик из make_genai_metric
    metrics = make_genai_metric(completion)
    mlflow.log_metrics(metrics)

    print("Ответ:", completion.choices[0].text.strip())



Ответ: Berlin.
🏃 View run crawling-finch-704 at: http://localhost:5000/#/experiments/506265846202140729/runs/3e500b562a4b4d31aabdf3b9850a080b
🧪 View experiment at: http://localhost:5000/#/experiments/506265846202140729


Скрин эксперимента - 2025-08-31_18-18.png